In [1]:
#%%
# =======================================================================
# CELL 0: ONE-SHOT SETUP (NO RESTART NEEDED)
# =======================================================================
print("="*70)
print("COMPLETE SETUP - ONE-SHOT VERSION")
print("="*70)

import subprocess
import sys

# Fix protobuf silently
print("[1/2] Fixing protobuf...")
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "protobuf"], 
               capture_output=True)
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "protobuf==3.20.3"], 
               capture_output=True)
print("      ✓ Protobuf fixed")

# Install packages silently
print("[2/2] Installing packages...")
subprocess.run([
    sys.executable, "-m", "pip", "install", "-q",
    "pandas", "numpy", "scikit-learn",
    "transformers", "torch", "accelerate", "peft",
    "hdbscan", "umap-learn"
], capture_output=True)
print("      ✓ Packages installed")

# Force reload protobuf
import importlib
import sys
if 'google.protobuf' in sys.modules:
    importlib.reload(sys.modules['google.protobuf'])

# Now import everything
import json
import pandas as pd
import numpy as np
import os
from collections import defaultdict, Counter
import torch
from transformers import T5EncoderModel, AutoTokenizer
from peft import PeftModel
from hdbscan import HDBSCAN
from sklearn.metrics import silhouette_score, silhouette_samples, davies_bouldin_score
import warnings
warnings.filterwarnings('ignore')

print("\n✅ All libraries imported successfully")

# GPU check
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ No GPU")

print("\n" + "="*70)
print("CELL 0: COMPLETE - Proceed to Cell 1")
print("="*70)

COMPLETE SETUP - ONE-SHOT VERSION
[1/2] Fixing protobuf...
      ✓ Protobuf fixed
[2/2] Installing packages...
      ✓ Packages installed


2025-11-09 07:50:05.836035: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762674606.203663      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762674606.317117      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



✅ All libraries imported successfully
✅ GPU: Tesla T4

CELL 0: COMPLETE - Proceed to Cell 1


In [2]:
# Verify protobuf version
import google.protobuf
print(f"✓ Protobuf version: {google.protobuf.__version__}")
if google.protobuf.__version__.startswith("3.20"):
    print("✅ Protobuf version is correct!")
else:
    print(f"⚠️ WARNING: Protobuf version is {google.protobuf.__version__}, expected 3.20.x")

✓ Protobuf version: 3.20.3
✅ Protobuf version is correct!


In [3]:
#%%
# =======================================================================
# CELL 1: LOAD DATA (BULLETPROOF NUMPY FIX)
# =======================================================================
print("="*70)
print("CELL 1: LOADING DATA")
print("="*70)

# Configuration
DATA_FILE_JSON = '/kaggle/input/cl-data-balanced/combined_data_new.json'
OUTPUT_CANDIDATES_FILE = 'candidates.json'
CONTEXT_WINDOW = 5

# Load data
print(f"\n[Step 1] Loading data...")
df = pd.read_json(DATA_FILE_JSON, orient='records')
print(f"  ✓ Loaded {len(df)} records")

print(f"\n[Step 2] Validating data...")
print(f"  Rows: {len(df)}")
print(f"  Columns: {df.columns.tolist()}")

# Process candidates
print(f"\n[Step 3] Processing {len(df)} records...")
candidates = []
errors_found = 0

for idx, row in df.iterrows():
    try:
        text = str(row.get('text', ''))
        aspect_term = str(row.get('aspect_term', ''))
        span_indices_raw = row.get('span_indices')
        is_implicit = bool(row.get('is_implicit', False))
        
        tokens = text.split()
        if not tokens:
            tokens = [""]

        span_text = ""
        span_indices = None
        
        # BULLETPROOF: Convert span_indices to usable format
        try:
            if span_indices_raw is None or pd.isna(span_indices_raw):
                span_indices = None
            elif isinstance(span_indices_raw, str):
                if span_indices_raw.strip().lower() in ['nan', 'null', 'none', '']:
                    span_indices = None
                else:
                    span_indices = json.loads(span_indices_raw.replace("'", "\""))
            elif isinstance(span_indices_raw, np.ndarray):
                # Convert numpy array to list
                span_indices = span_indices_raw.tolist()
            elif isinstance(span_indices_raw, (list, tuple)):
                span_indices = list(span_indices_raw)
            else:
                span_indices = None
        except:
            span_indices = None
        
        # Now check if we have valid span_indices
        if span_indices is not None and isinstance(span_indices, list) and len(span_indices) >= 2:
            try:
                span_start = int(span_indices[0])
                span_end = int(span_indices[1])
                
                # Bounds check
                span_start = max(0, min(span_start, len(tokens) - 1))
                span_end = max(span_start, min(span_end, len(tokens) - 1))
                
                span_text = " ".join(tokens[span_start : span_end + 1])
                
                # Create context window
                context_start = max(0, span_start - CONTEXT_WINDOW)
                context_end = min(len(tokens), span_end + 1 + CONTEXT_WINDOW)
                context = " ".join(tokens[context_start:context_end])
            except:
                # Fallback
                span_text = aspect_term.replace("_", " ")
                context = text
        else:
            # No valid span - use aspect_term
            span_text = aspect_term.replace("_", " ")
            context = text
        
        span_with_context = f"{span_text} [SEP] {context}"
        
        candidate = {
            "id": int(row.get('id', idx)),
            "text": text,
            "aspect_term": aspect_term,
            "span_text": span_text,
            "span_with_context": span_with_context,
            "is_implicit": is_implicit,
            "sentiment": str(row.get('sentiment', 'neutral'))
        }
        candidates.append(candidate)
        
    except Exception as e:
        errors_found += 1
        if errors_found <= 10:
            print(f"  ⚠️ Error at row {idx}: {e}")

print(f"\n  ✓ Processed: {len(candidates)} candidates")
if errors_found > 0:
    print(f"  ⚠️ Total errors: {errors_found}")

# Save
with open(OUTPUT_CANDIDATES_FILE, 'w', encoding='utf-8') as f:
    json.dump(candidates, f, indent=2)
print(f"  ✓ Saved to: {OUTPUT_CANDIDATES_FILE}")

# Statistics
print(f"\n[Step 4] Final Statistics:")
print(f"  Total: {len(candidates)}")

if len(candidates) > 0:
    implicit = sum(1 for c in candidates if c['is_implicit'])
    print(f"  Implicit: {implicit} ({implicit/len(candidates)*100:.1f}%)")
    
    aspects = Counter(c['aspect_term'] for c in candidates)
    print(f"  Unique aspects: {len(aspects)}")
    print(f"  Top 5: {aspects.most_common(5)}")
    
    print(f"\n  Sample:")
    s = candidates[0]
    print(f"    ID: {s['id']}")
    print(f"    Text: {s['text'][:50]}...")
    print(f"    Aspect: {s['aspect_term']}")
    print(f"    Span: {s['span_text']}")

print("\n" + "="*70)
print("CELL 1: COMPLETE ✅")
print("="*70)
print(f"✅ {len(candidates)} candidates ready")
print("➡️  Next: Run Cell 2 (embeddings - 5-10 min)\n")

CELL 1: LOADING DATA

[Step 1] Loading data...
  ✓ Loaded 1193 records

[Step 2] Validating data...
  Rows: 1193
  Columns: ['id', 'text', 'aspect_term', 'sentiment', 'span_indices', 'rationale', 'is_implicit', 'is_sarcastic', 'has_slang', 'has_emoji']

[Step 3] Processing 1193 records...

  ✓ Processed: 1193 candidates
  ✓ Saved to: candidates.json

[Step 4] Final Statistics:
  Total: 1193
  Implicit: 319 (26.7%)
  Unique aspects: 89
  Top 5: [('performance', 33), ('features', 27), ('acting', 26), ('design', 24), ('atmosphere', 24)]

  Sample:
    ID: 1
    Text: ugh my phone's battery is draining way too quick t...
    Aspect: battery_drain
    Span: battery drain

CELL 1: COMPLETE ✅
✅ 1193 candidates ready
➡️  Next: Run Cell 2 (embeddings - 5-10 min)



In [4]:
#%%
# =======================================================================
# CELL 2: GENERATE EMBEDDINGS (CORRECT PEFT LOADING)
# =======================================================================
print("="*70)
print("CELL 2: GENERATING EMBEDDINGS - FIXED WITH PEFT")
print("="*70)
print(f"Started: {pd.Timestamp.now(tz='UTC')}")
print("Estimated time: 5-10 minutes\n")

# Configuration
DAPT_ADAPTER_DIR = "/kaggle/input/t5-base-700k-masked-span/flan-t5-base-social-lora-masked-span"
BASE_MODEL_NAME = "google/flan-t5-base"
EMBEDDINGS_FILE = "embeddings.npy"
CANDIDATES_FILE = "candidates.json"

# Load candidates
print("[Step 1] Loading candidates...")
with open(CANDIDATES_FILE, 'r', encoding='utf-8') as f:
    candidates = json.load(f)
print(f"  ✓ Loaded {len(candidates)} candidates\n")

# Load model (CORRECT WAY - use T5ForConditionalGeneration)
print("[Step 2] Loading model with PEFT adapter...")

from transformers import T5ForConditionalGeneration, AutoTokenizer
from peft import PeftModel, PeftConfig

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
print(f"  ✓ Tokenizer loaded")

# Load PEFT config (sanity check)
try:
    peft_config = PeftConfig.from_pretrained(DAPT_ADAPTER_DIR)
    print(f"  ✓ PEFT config loaded")
    print(f"    Task type: {peft_config.task_type}")
except Exception as e:
    print(f"  ⚠️  Could not load PEFT config: {e}")

# Load FULL seq2seq model (this is the key!)
print(f"  Loading T5ForConditionalGeneration base model...")
base_model = T5ForConditionalGeneration.from_pretrained(BASE_MODEL_NAME)
print(f"  ✓ Base model loaded")

# Attach PEFT adapter
print(f"  Loading adapter from: {DAPT_ADAPTER_DIR}")
try:
    model_with_adapter = PeftModel.from_pretrained(base_model, DAPT_ADAPTER_DIR)
    print(f"  ✓ PEFT adapter attached")
    
    # Merge adapter into base model
    print(f"  Merging adapter weights...")
    model = model_with_adapter.merge_and_unload()
    print(f"  ✓ Adapter merged successfully!")
    adapter_loaded = True
    
except Exception as e:
    print(f"  ⚠️  Adapter loading failed: {e}")
    print(f"  Using base model WITHOUT adapter")
    model = base_model
    adapter_loaded = False

# Move to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

print(f"\n  ✓ Model on: {device}")
if torch.cuda.is_available():
    print(f"  ✓ GPU: {torch.cuda.get_device_name(0)}")

if adapter_loaded:
    print(f"  ✅ DAPT adapter is ACTIVE")
else:
    print(f"  ⚠️  Using base model only (no adapter)")

# Embedding function (use encoder from seq2seq model)
def embed_batch(batch_texts):
    inputs = tokenizer(
        batch_texts,
        return_tensors="pt",
        max_length=128,
        truncation=True,
        padding="max_length"
    ).to(device)
    
    with torch.no_grad():
        # Get encoder from the seq2seq model
        encoder = model.get_encoder()
        encoder_outputs = encoder(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask']
        )
        hidden_states = encoder_outputs.last_hidden_state
        
        # Masked mean pooling
        attention_mask = inputs['attention_mask'].unsqueeze(-1)
        masked_hidden = hidden_states * attention_mask
        sum_hidden = masked_hidden.sum(dim=1)
        sum_mask = attention_mask.sum(dim=1).clamp(min=1e-9)
        embedding = sum_hidden / sum_mask
        
        # L2 normalize
        embedding = embedding / torch.norm(embedding, dim=1, keepdim=True).clamp(min=1e-9)
    
    return embedding.cpu().numpy()

# Generate embeddings
print(f"\n[Step 3] Generating embeddings...")
batch_size = 32
texts_to_embed = [c['span_with_context'] for c in candidates]

print(f"  Total: {len(texts_to_embed)}")
print(f"  Batch size: {batch_size}\n")

all_embeddings = []
for i in range(0, len(texts_to_embed), batch_size):
    batch_texts = texts_to_embed[i:i + batch_size]
    batch_embeddings = embed_batch(batch_texts)
    all_embeddings.append(batch_embeddings)
    
    if (i // batch_size) % 10 == 0 or i + batch_size >= len(texts_to_embed):
        progress = min(i + batch_size, len(texts_to_embed))
        print(f"  {progress:4d} / {len(texts_to_embed)} ({progress/len(texts_to_embed)*100:5.1f}%)")

embeddings_matrix = np.vstack(all_embeddings)

# Save
np.save(EMBEDDINGS_FILE, embeddings_matrix)
print(f"\n✓ Saved: {EMBEDDINGS_FILE}")
print(f"✓ Shape: {embeddings_matrix.shape}")

# Quality check
print(f"\n[Step 4] Quality check:")
print(f"  NaN: {np.isnan(embeddings_matrix).any()}")
print(f"  Inf: {np.isinf(embeddings_matrix).any()}")

if len(embeddings_matrix) >= 3:
    sim_01 = np.dot(embeddings_matrix[0], embeddings_matrix[1])
    sim_02 = np.dot(embeddings_matrix[0], embeddings_matrix[2])
    print(f"  Sample similarities:")
    print(f"    [0]·[1]: {sim_01:.3f}")
    print(f"    [0]·[2]: {sim_02:.3f}")

# Cleanup
del model
del base_model
if 'model_with_adapter' in locals():
    del model_with_adapter
torch.cuda.empty_cache()
print(f"\n  ✓ GPU memory cleared")

print("\n" + "="*70)
print("CELL 2: COMPLETE ✅")
print("="*70)
print(f"Completed: {pd.Timestamp.now(tz='UTC')}")

if adapter_loaded:
    print("✅ DAPT adapter was successfully used for embeddings")
else:
    print("⚠️  Base model used (adapter not loaded)")

print("\n➡️  Next: Run Cell 3 (clustering)\n")

CELL 2: GENERATING EMBEDDINGS - FIXED WITH PEFT
Started: 2025-11-09 07:50:36.936648+00:00
Estimated time: 5-10 minutes

[Step 1] Loading candidates...
  ✓ Loaded 1193 candidates

[Step 2] Loading model with PEFT adapter...


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

  ✓ Tokenizer loaded
  ✓ PEFT config loaded
    Task type: SEQ_2_SEQ_LM
  Loading T5ForConditionalGeneration base model...


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

  ✓ Base model loaded
  Loading adapter from: /kaggle/input/t5-base-700k-masked-span/flan-t5-base-social-lora-masked-span
  ✓ PEFT adapter attached
  Merging adapter weights...
  ✓ Adapter merged successfully!

  ✓ Model on: cuda
  ✓ GPU: Tesla T4
  ✅ DAPT adapter is ACTIVE

[Step 3] Generating embeddings...
  Total: 1193
  Batch size: 32

    32 / 1193 (  2.7%)
   352 / 1193 ( 29.5%)
   672 / 1193 ( 56.3%)
   992 / 1193 ( 83.2%)
  1193 / 1193 (100.0%)

✓ Saved: embeddings.npy
✓ Shape: (1193, 768)

[Step 4] Quality check:
  NaN: False
  Inf: False
  Sample similarities:
    [0]·[1]: 0.809
    [0]·[2]: 0.684

  ✓ GPU memory cleared

CELL 2: COMPLETE ✅
Completed: 2025-11-09 07:50:51.869995+00:00
✅ DAPT adapter was successfully used for embeddings

➡️  Next: Run Cell 3 (clustering)



In [5]:
#%%
# =======================================================================
# CELL 3: HDBSCAN CLUSTERING
# =======================================================================
print("="*70)
print("CELL 3: HDBSCAN CLUSTERING")
print("="*70)
print(f"Time: {pd.Timestamp.now(tz='UTC')}\n")

EMBEDDINGS_FILE = "embeddings.npy"
CANDIDATES_FILE = "candidates.json"

# Load data
print("[Step 1] Loading data...")
embeddings_matrix = np.load(EMBEDDINGS_FILE)
with open(CANDIDATES_FILE, 'r') as f:
    candidates = json.load(f)

print(f"  ✓ Embeddings: {embeddings_matrix.shape}")
print(f"  ✓ Candidates: {len(candidates)}\n")

# Test multiple HDBSCAN configurations
print("[Step 2] Testing HDBSCAN configurations...\n")

configs = [
    {"name": "Baseline", "min_cluster_size": 3, "min_samples": 2, "method": "eom"},
    {"name": "Lenient", "min_cluster_size": 2, "min_samples": 1, "method": "eom"},
    {"name": "Aggressive", "min_cluster_size": 2, "min_samples": 1, "method": "leaf"},
]

results = []

for cfg in configs:
    print(f"  [{cfg['name']}]")
    print(f"    min_cluster_size={cfg['min_cluster_size']}, min_samples={cfg['min_samples']}, method={cfg['method']}")
    
    clusterer = HDBSCAN(
        min_cluster_size=cfg['min_cluster_size'],
        min_samples=cfg['min_samples'],
        metric='euclidean',
        cluster_selection_method=cfg['method'],
        prediction_data=True
    )
    
    labels = clusterer.fit_predict(embeddings_matrix)
    
    num_clusters = labels.max() + 1
    noise_count = (labels == -1).sum()
    noise_pct = noise_count / len(labels) * 100
    
    print(f"    → Clusters: {num_clusters}, Noise: {noise_count} ({noise_pct:.1f}%)\n")
    
    results.append({
        "name": cfg['name'],
        "labels": labels,
        "clusterer": clusterer,
        "num_clusters": num_clusters,
        "noise_pct": noise_pct,
        "noise_count": noise_count
    })

# Select best configuration
print("[Step 3] Selecting best configuration...")

def score_config(r):
    """Score clustering quality (lower is better)"""
    # Heavily penalize high noise
    noise_penalty = r['noise_pct'] ** 1.5
    # Penalize deviation from target range (60-90 clusters)
    cluster_penalty = abs(r['num_clusters'] - 75) * 0.5
    return noise_penalty + cluster_penalty

for r in results:
    r['score'] = score_config(r)
    print(f"  {r['name']:12s}: score={r['score']:6.1f} ({r['num_clusters']:3d} clusters, {r['noise_pct']:5.1f}% noise)")

best = min(results, key=lambda x: x['score'])

print(f"\n  ✅ Selected: {best['name']}")
print(f"     Clusters: {best['num_clusters']}")
print(f"     Noise: {best['noise_count']} ({best['noise_pct']:.1f}%)")

# Quality gates
print(f"\n[Step 4] Quality assessment:")
if best['num_clusters'] < 60:
    print(f"  ⚠️  Few clusters ({best['num_clusters']} < 60 target)")
elif best['num_clusters'] > 150:
    print(f"  ⚠️  Many clusters ({best['num_clusters']} > 150)")
else:
    print(f"  ✅ Cluster count in acceptable range (60-150)")

if best['noise_pct'] < 20:
    print(f"  ✅ Low noise ({best['noise_pct']:.1f}% < 20% target)")
elif best['noise_pct'] < 30:
    print(f"  ⚠️  Moderate noise ({best['noise_pct']:.1f}%)")
else:
    print(f"  ❌ High noise ({best['noise_pct']:.1f}% > 30%)")

# Save best results
labels = best['labels']
probabilities = best['clusterer'].probabilities_

np.save("cluster_labels.npy", labels)
np.save("cluster_probabilities.npy", probabilities)

print(f"\n  ✓ Saved cluster_labels.npy")
print(f"  ✓ Saved cluster_probabilities.npy")

print("\n" + "="*70)
print("CELL 3: COMPLETE ✅")
print("="*70)
print(f"Final: {best['num_clusters']} clusters, {best['noise_pct']:.1f}% noise")
print(f"Time: {pd.Timestamp.now(tz='UTC')}")
print("\n➡️  Next: Run Cell 4 (cluster statistics)\n")

CELL 3: HDBSCAN CLUSTERING
Time: 2025-11-09 07:50:51.906748+00:00

[Step 1] Loading data...
  ✓ Embeddings: (1193, 768)
  ✓ Candidates: 1193

[Step 2] Testing HDBSCAN configurations...

  [Baseline]
    min_cluster_size=3, min_samples=2, method=eom
    → Clusters: 145, Noise: 474 (39.7%)

  [Lenient]
    min_cluster_size=2, min_samples=1, method=eom
    → Clusters: 359, Noise: 244 (20.5%)

  [Aggressive]
    min_cluster_size=2, min_samples=1, method=leaf
    → Clusters: 371, Noise: 252 (21.1%)

[Step 3] Selecting best configuration...
  Baseline    : score= 285.4 (145 clusters,  39.7% noise)
  Lenient     : score= 234.5 (359 clusters,  20.5% noise)
  Aggressive  : score= 245.1 (371 clusters,  21.1% noise)

  ✅ Selected: Lenient
     Clusters: 359
     Noise: 244 (20.5%)

[Step 4] Quality assessment:
  ⚠️  Many clusters (359 > 150)
  ⚠️  Moderate noise (20.5%)

  ✓ Saved cluster_labels.npy
  ✓ Saved cluster_probabilities.npy

CELL 3: COMPLETE ✅
Final: 359 clusters, 20.5% noise
Time: 202

In [6]:
#%%
# =======================================================================
# CELL 4: COMPUTE CLUSTER STATISTICS
# =======================================================================
print("="*70)
print("CELL 4: COMPUTING CLUSTER STATISTICS")
print("="*70)
print(f"Time: {pd.Timestamp.now(tz='UTC')}\n")

# Load data
print("[Step 1] Loading data...")
with open('candidates.json', 'r') as f:
    candidates = json.load(f)
embeddings_matrix = np.load('embeddings.npy')
labels = np.load('cluster_labels.npy')
probabilities = np.load('cluster_probabilities.npy')

print(f"  ✓ Loaded {len(candidates)} candidates")
print(f"  ✓ Loaded {len(labels)} labels\n")

# Build cluster memberships
print("[Step 2] Building cluster memberships...")
clusters = defaultdict(list)

for idx, label in enumerate(labels):
    if label != -1:  # Skip noise
        clusters[label].append({
            "idx": idx,
            "candidate": candidates[idx],
            "probability": probabilities[idx]
        })

print(f"  ✓ Built {len(clusters)} non-noise clusters\n")

# Compute statistics for each cluster
print("[Step 3] Computing cluster statistics...")
cluster_stats = []

for cluster_id, members in clusters.items():
    member_indices = [m["idx"] for m in members]
    cluster_embeddings = embeddings_matrix[member_indices]
    
    # Centroid
    centroid = cluster_embeddings.mean(axis=0)
    
    # Variance (cohesion measure)
    distances = np.linalg.norm(cluster_embeddings - centroid, axis=1)
    variance = distances.var()
    
    # Representatives (5 nearest to centroid)
    top_indices = np.argsort(distances)[:5]
    representatives = [members[i]["candidate"] for i in top_indices]
    
    # Aspect term distribution
    aspect_terms = [m["candidate"]["aspect_term"] for m in members]
    aspect_counter = Counter(aspect_terms)
    
    # Sentiment distribution
    sentiments = [m["candidate"]["sentiment"] for m in members]
    sentiment_counter = Counter(sentiments)
    
    # Implicit ratio
    implicit_count = sum(1 for m in members if m["candidate"].get("is_implicit", False))
    implicit_ratio = implicit_count / len(members)
    
    cluster_stats.append({
        "cluster_id": int(cluster_id),
        "size": len(members),
        "variance": float(variance),
        "centroid": centroid.tolist(),
        "representatives": representatives,
        "aspect_terms": aspect_counter.most_common(5),
        "sentiment_distribution": dict(sentiment_counter),
        "implicit_ratio": float(implicit_ratio),
        "avg_probability": float(np.mean([m["probability"] for m in members])),
        "member_indices": member_indices
    })

# Sort by size (largest first)
cluster_stats.sort(key=lambda x: x["size"], reverse=True)

print(f"  ✓ Computed statistics for {len(cluster_stats)} clusters\n")

# Save to file
print("[Step 4] Saving cluster statistics...")
with open("cluster_stats.json", "w") as f:
    json.dump(cluster_stats, f, indent=2)

print(f"  ✓ Saved cluster_stats.json\n")

# Summary statistics
print("[Step 5] Summary:")
sizes = [s["size"] for s in cluster_stats]
variances = [s["variance"] for s in cluster_stats]

print(f"  Total clusters: {len(cluster_stats)}")
print(f"  Total members: {sum(sizes)} (excludes {(labels == -1).sum()} noise points)")
print(f"\n  Cluster size distribution:")
print(f"    Min:    {min(sizes)}")
print(f"    Max:    {max(sizes)}")
print(f"    Mean:   {np.mean(sizes):.1f}")
print(f"    Median: {np.median(sizes):.0f}")
print(f"\n  Cluster variance distribution:")
print(f"    Mean:   {np.mean(variances):.3f}")
print(f"    Median: {np.median(variances):.3f}")

# Show top 10 largest clusters
print(f"\n  Top 10 largest clusters:")
for i, s in enumerate(cluster_stats[:10], 1):
    print(f"  {i:2d}. Cluster {s['cluster_id']:3d}: size={s['size']:3d}, variance={s['variance']:.3f}")
    print(f"      Top aspects: {[f'{term}({count})' for term, count in s['aspect_terms'][:3]]}")
    print(f"      Implicit: {s['implicit_ratio']*100:.0f}%")
    print(f"      Sample: '{s['representatives'][0]['span_text']}'")

# Identify potential issues
print(f"\n  Cluster health check:")
very_large = [s for s in cluster_stats if s['size'] > 50]
very_small = [s for s in cluster_stats if s['size'] == 2]
high_variance = [s for s in cluster_stats if s['variance'] > 1.0]

if very_large:
    print(f"    ⚠️  {len(very_large)} very large clusters (>50 members) - may be garbage bins")
if very_small:
    print(f"    ⚠️  {len(very_small)} size-2 clusters - consider merging")
if high_variance:
    print(f"    ⚠️  {len(high_variance)} high-variance clusters (>1.0) - may be incoherent")

if not very_large and not high_variance:
    print(f"    ✅ No major structural issues detected")

print("\n" + "="*70)
print("CELL 4: COMPLETE ✅")
print("="*70)
print(f"Time: {pd.Timestamp.now(tz='UTC')}")
print("\n➡️  Next: Run Cell 5 (validation metrics)\n")

CELL 4: COMPUTING CLUSTER STATISTICS
Time: 2025-11-09 07:50:59.434261+00:00

[Step 1] Loading data...
  ✓ Loaded 1193 candidates
  ✓ Loaded 1193 labels

[Step 2] Building cluster memberships...
  ✓ Built 359 non-noise clusters

[Step 3] Computing cluster statistics...
  ✓ Computed statistics for 359 clusters

[Step 4] Saving cluster statistics...
  ✓ Saved cluster_stats.json

[Step 5] Summary:
  Total clusters: 359
  Total members: 949 (excludes 244 noise points)

  Cluster size distribution:
    Min:    2
    Max:    11
    Mean:   2.6
    Median: 2

  Cluster variance distribution:
    Mean:   0.001
    Median: 0.000

  Top 10 largest clusters:
   1. Cluster  33: size= 11, variance=0.000
      Top aspects: ['battery_life(1)', 'sound_quality(1)', 'design(1)']
      Implicit: 9%
      Sample: 'performance'
   2. Cluster  60: size=  9, variance=0.000
      Top aspects: ['plot(1)', 'acting(1)', 'pacing(1)']
      Implicit: 11%
      Sample: 'design'
   3. Cluster  39: size=  9, variance=

In [7]:
#%%
# =======================================================================
# CELL 5: VALIDATION METRICS
# =======================================================================
print("="*70)
print("CELL 5: COMPUTING VALIDATION METRICS")
print("="*70)
print(f"Time: {pd.Timestamp.now(tz='UTC')}\n")

# Load data
print("[Step 1] Loading data...")
embeddings_matrix = np.load('embeddings.npy')
labels = np.load('cluster_labels.npy')
probabilities = np.load('cluster_probabilities.npy')

print(f"  ✓ Loaded embeddings: {embeddings_matrix.shape}")
print(f"  ✓ Loaded labels: {len(labels)}\n")

# Filter out noise points for metrics
print("[Step 2] Filtering noise points...")
non_noise_mask = labels != -1
filtered_embeddings = embeddings_matrix[non_noise_mask]
filtered_labels = labels[non_noise_mask]

num_noise = (labels == -1).sum()
num_clustered = len(filtered_labels)
num_unique_clusters = len(np.unique(filtered_labels))

print(f"  ✓ Non-noise points: {num_clustered}")
print(f"  ✓ Noise points: {num_noise}")
print(f"  ✓ Unique clusters: {num_unique_clusters}\n")

# Compute metrics (only if we have enough clusters)
if num_unique_clusters > 1:
    
    # Silhouette Score
    print("[Step 3] Computing Silhouette Score...")
    print("  (This may take 1-2 minutes for 949 points...)")
    overall_silhouette = silhouette_score(
        filtered_embeddings, 
        filtered_labels, 
        metric='euclidean',
        sample_size=min(1000, len(filtered_labels))  # Sample for speed
    )
    print(f"  ✓ Overall Silhouette: {overall_silhouette:.3f}\n")
    
    # Davies-Bouldin Index
    print("[Step 4] Computing Davies-Bouldin Index...")
    db_index = davies_bouldin_score(filtered_embeddings, filtered_labels)
    print(f"  ✓ Davies-Bouldin: {db_index:.3f}\n")
    
    # Per-cluster Silhouette (for identifying bad clusters)
    print("[Step 5] Computing per-cluster Silhouette...")
    sample_silhouettes = silhouette_samples(
        filtered_embeddings, 
        filtered_labels, 
        metric='euclidean'
    )
    
    cluster_silhouettes = {}
    for cid in np.unique(filtered_labels):
        mask = filtered_labels == cid
        cluster_silhouettes[int(cid)] = sample_silhouettes[mask].mean()
    
    # Find problematic clusters (low silhouette)
    low_sil_clusters = [(cid, score) for cid, score in cluster_silhouettes.items() if score < 0.2]
    low_sil_clusters.sort(key=lambda x: x[1])
    
    print(f"  ✓ Per-cluster silhouettes computed")
    print(f"  Clusters with silhouette < 0.2: {len(low_sil_clusters)}")
    
    if low_sil_clusters:
        print(f"\n  Worst 10 clusters (lowest silhouette):")
        for cid, score in low_sil_clusters[:10]:
            print(f"    Cluster {cid:3d}: {score:6.3f}")
    
    # Overall assessment
    print("\n" + "="*70)
    print("QUALITY ASSESSMENT")
    print("="*70)
    
    print(f"\n1. Silhouette Score: {overall_silhouette:.3f}")
    print(f"   Interpretation:")
    if overall_silhouette >= 0.50:
        print(f"   ✅ EXCELLENT (≥0.50) - Strong cluster structure")
    elif overall_silhouette >= 0.30:
        print(f"   ✅ GOOD (0.30-0.49) - Acceptable cluster structure")
    elif overall_silhouette >= 0.20:
        print(f"   ⚠️  MARGINAL (0.20-0.29) - Weak structure, proceed with caution")
    else:
        print(f"   ❌ POOR (<0.20) - Clusters may not be meaningful")
    
    print(f"\n2. Davies-Bouldin Index: {db_index:.3f}")
    print(f"   Interpretation (lower is better):")
    if db_index < 1.0:
        print(f"   ✅ GOOD (<1.0) - Well-separated clusters")
    elif db_index < 1.5:
        print(f"   ✅ ACCEPTABLE (1.0-1.5) - Moderate separation")
    elif db_index < 2.0:
        print(f"   ⚠️  MARGINAL (1.5-2.0) - Some cluster overlap")
    else:
        print(f"   ❌ POOR (≥2.0) - High overlap between clusters")
    
    print(f"\n3. Cluster Count: {num_unique_clusters}")
    print(f"   Target: 60-150 clusters")
    if num_unique_clusters > 150:
        print(f"   ⚠️  HIGH - Many clusters (consider post-processing merge)")
    elif num_unique_clusters < 60:
        print(f"   ⚠️  LOW - Few clusters")
    else:
        print(f"   ✅ OPTIMAL - Good cluster count")
    
    print(f"\n4. Noise Ratio: {num_noise / len(labels) * 100:.1f}%")
    print(f"   Target: <20%")
    if num_noise / len(labels) < 0.20:
        print(f"   ✅ LOW - Good clustering coverage")
    elif num_noise / len(labels) < 0.30:
        print(f"   ⚠️  MODERATE - Acceptable")
    else:
        print(f"   ❌ HIGH - Too many unassigned points")
    
    # Overall verdict
    print(f"\n" + "="*70)
    print("OVERALL VERDICT")
    print("="*70)
    
    verdict_score = 0
    if overall_silhouette >= 0.30: verdict_score += 1
    if db_index < 1.5: verdict_score += 1
    if 60 <= num_unique_clusters <= 200: verdict_score += 1
    if num_noise / len(labels) < 0.30: verdict_score += 1
    
    if verdict_score >= 3:
        print("✅ PASS - Clustering quality is acceptable for Stage 1")
        print("   You can proceed to contrastive learning.")
    elif verdict_score >= 2:
        print("⚠️  MARGINAL - Clustering is usable but not ideal")
        print("   Consider post-processing or re-clustering with different params.")
    else:
        print("❌ FAIL - Clustering quality is poor")
        print("   Review data quality and consider different approaches.")
    
    print(f"\nScore: {verdict_score}/4 quality gates passed")
    
else:
    print("⚠️  Not enough clusters (≤1) to compute metrics")

print("\n" + "="*70)
print("CELL 5: COMPLETE ✅")
print("="*70)
print(f"Time: {pd.Timestamp.now(tz='UTC')}")
print("\n➡️  Next: Run Cell 6 (manual validation sample)\n")

CELL 5: COMPUTING VALIDATION METRICS
Time: 2025-11-09 07:50:59.870983+00:00

[Step 1] Loading data...
  ✓ Loaded embeddings: (1193, 768)
  ✓ Loaded labels: 1193

[Step 2] Filtering noise points...
  ✓ Non-noise points: 949
  ✓ Noise points: 244
  ✓ Unique clusters: 359

[Step 3] Computing Silhouette Score...
  (This may take 1-2 minutes for 949 points...)
  ✓ Overall Silhouette: 0.568

[Step 4] Computing Davies-Bouldin Index...
  ✓ Davies-Bouldin: 0.893

[Step 5] Computing per-cluster Silhouette...
  ✓ Per-cluster silhouettes computed
  Clusters with silhouette < 0.2: 101

  Worst 10 clusters (lowest silhouette):
    Cluster 103: -0.041
    Cluster 140: -0.011
    Cluster  94: -0.003
    Cluster 311: -0.003
    Cluster 158:  0.015
    Cluster 193:  0.021
    Cluster  87:  0.025
    Cluster 118:  0.029
    Cluster 207:  0.037
    Cluster 189:  0.038

QUALITY ASSESSMENT

1. Silhouette Score: 0.568
   Interpretation:
   ✅ EXCELLENT (≥0.50) - Strong cluster structure

2. Davies-Bouldin Ind

In [8]:
#%%
# =======================================================================
# CELL 6: MANUAL VALIDATION SAMPLE
# =======================================================================
print("="*70)
print("CELL 6: MANUAL VALIDATION SAMPLE")
print("="*70)
print("Inspect these samples to verify semantic coherence\n")

import random

# Load cluster stats
with open('cluster_stats.json', 'r') as f:
    cluster_stats = json.load(f)

print(f"Total clusters: {len(cluster_stats)}\n")

# Sampling strategy
small = [s for s in cluster_stats if s["size"] == 2]
medium = [s for s in cluster_stats if 3 <= s["size"] <= 5]
large = [s for s in cluster_stats if s["size"] >= 6]

print(f"Cluster size breakdown:")
print(f"  Size 2: {len(small)}")
print(f"  Size 3-5: {len(medium)}")
print(f"  Size 6+: {len(large)}\n")

# Sample clusters
random.seed(42)
sample = (
    random.sample(small, min(5, len(small))) +
    random.sample(medium, min(10, len(medium))) +
    random.sample(large, min(10, len(large)))
)

print("="*70)
print("MANUAL VALIDATION SAMPLES")
print("="*70)
print("Review these clusters for semantic coherence:\n")

for i, stat in enumerate(sample, 1):
    print(f"\n{'='*70}")
    print(f"SAMPLE {i}: Cluster {stat['cluster_id']} (Size: {stat['size']}, Variance: {stat['variance']:.4f})")
    print(f"{'='*70}")
    
    # Top aspects
    top_aspects = stat['aspect_terms'][:3]
    print(f"\nTop aspects: {top_aspects}")
    
    # Sentiment distribution
    if 'sentiment_distribution' in stat:
        print(f"Sentiments: {stat['sentiment_distribution']}")
    
    # Implicit ratio
    if 'implicit_ratio' in stat:
        print(f"Implicit ratio: {stat['implicit_ratio']*100:.0f}%")
    
    # Show representatives
    print(f"\nRepresentative samples:")
    for j, rep in enumerate(stat['representatives'][:5], 1):
        text = rep['text']
        if len(text) > 80:
            text = text[:80] + "..."
        print(f"\n  {j}. Text: \"{text}\"")
        print(f"     Span: '{rep['span_text']}' | Aspect: '{rep['aspect_term']}' | Sentiment: {rep['sentiment']}")
    
    print(f"\n  ✓ Semantically coherent? (Check manually)")
    print(f"  ✓ Good paraphrases/synonyms? (Check manually)")

print("\n" + "="*70)
print("MANUAL VALIDATION CHECKLIST")
print("="*70)
print("""
Review the samples above and check:

1. SEMANTIC COHERENCE
   ✓ Do all members discuss the same underlying concept?
   ✓ Would a human group these together?

2. SURFACE FORM DIVERSITY
   ✓ Do members use different words/phrases for the same thing?
   ✓ Are they good paraphrases (not exact duplicates)?

3. IMPLICIT/EXPLICIT MIX
   ✓ Does the cluster contain both implicit and explicit mentions?
   ✓ This is GOOD - shows the model generalizes beyond surface forms

TARGET: At least 70% of sampled clusters should be semantically coherent.

If most clusters look good → Proceed to Cell 7 (final export)
If many clusters are incoherent → Consider re-running with different params
""")

print("="*70)
print("CELL 6: COMPLETE ✅")
print("="*70)
print("\n➡️  Next: Run Cell 7 (final export)\n")

CELL 6: MANUAL VALIDATION SAMPLE
Inspect these samples to verify semantic coherence

Total clusters: 359

Cluster size breakdown:
  Size 2: 227
  Size 3-5: 120
  Size 6+: 12

MANUAL VALIDATION SAMPLES
Review these clusters for semantic coherence:


SAMPLE 1: Cluster 214 (Size: 2, Variance: 0.0000)

Top aspects: [['performance', 2]]
Sentiments: {'positive': 2}
Implicit ratio: 0%

Representative samples:

  1. Text: "the whole setup with the new router, performance is god tier but setup took 3 ho..."
     Span: 'performance' | Aspect: 'performance' | Sentiment: positive

  2. Text: "the whole setup with the new router, performance is god tier but setup took 3 ho..."
     Span: 'performance' | Aspect: 'performance' | Sentiment: positive

  ✓ Semantically coherent? (Check manually)
  ✓ Good paraphrases/synonyms? (Check manually)

SAMPLE 2: Cluster 234 (Size: 2, Variance: 0.0000)

Top aspects: [['gym_experience', 2]]
Sentiments: {'positive': 2}
Implicit ratio: 50%

Representative samples:



In [9]:
#%%
# =======================================================================
# DUPLICATE CHECK
# =======================================================================
print("="*70)
print("CHECKING FOR DUPLICATES IN SOURCE DATA")
print("="*70)

with open('candidates.json', 'r') as f:
    candidates = json.load(f)

print(f"\nTotal candidates: {len(candidates)}\n")

# Check for exact duplicates
from collections import Counter

# Check duplicate texts
texts = [c['text'] for c in candidates]
text_counts = Counter(texts)
duplicates = {text: count for text, count in text_counts.items() if count > 1}

print(f"[1] Duplicate texts:")
print(f"  Unique texts: {len(text_counts)}")
print(f"  Texts appearing multiple times: {len(duplicates)}")

if duplicates:
    print(f"\n  Top 10 most duplicated texts:")
    for text, count in sorted(duplicates.items(), key=lambda x: x[1], reverse=True)[:10]:
        print(f"    {count}x: {text[:60]}...")

# Check duplicate text+aspect combinations
text_aspect_pairs = [(c['text'], c['aspect_term']) for c in candidates]
pair_counts = Counter(text_aspect_pairs)
dup_pairs = {pair: count for pair, count in pair_counts.items() if count > 1}

print(f"\n[2] Duplicate text+aspect_term combinations:")
print(f"  Unique combinations: {len(pair_counts)}")
print(f"  Duplicated combinations: {len(dup_pairs)}")

if dup_pairs:
    print(f"\n  Sample duplicated pairs:")
    for (text, aspect), count in list(dup_pairs.items())[:5]:
        print(f"    {count}x: Text='{text[:40]}...' | Aspect='{aspect}'")

# Check IDs
ids = [c['id'] for c in candidates]
id_counts = Counter(ids)
dup_ids = {id_: count for id_, count in id_counts.items() if count > 1}

print(f"\n[3] Duplicate IDs:")
print(f"  Unique IDs: {len(id_counts)}")
print(f"  Duplicated IDs: {len(dup_ids)}")

if dup_ids:
    print(f"  Sample duplicated IDs:")
    for id_, count in list(dup_ids.items())[:10]:
        print(f"    ID {id_}: appears {count} times")

# Check exact record duplicates (all fields)
import json
record_strings = [json.dumps(c, sort_keys=True) for c in candidates]
record_counts = Counter(record_strings)
exact_dups = sum(1 for count in record_counts.values() if count > 1)

print(f"\n[4] Exact record duplicates (all fields identical):")
print(f"  Exact duplicates: {exact_dups}")

print("\n" + "="*70)
print("DUPLICATE CHECK COMPLETE")
print("="*70)

CHECKING FOR DUPLICATES IN SOURCE DATA

Total candidates: 1193

[1] Duplicate texts:
  Unique texts: 811
  Texts appearing multiple times: 242

  Top 10 most duplicated texts:
    13x: signed up for the streaming service trial and binged three s...
    12x: went to that new fusion spot downtown last night with the cr...
    12x: debating the latest bill in congress all week, leadership pu...
    12x: upgraded to the latest smartphone last week and its been a m...
    12x: finally dove into that new rpg over the weekend and it sucke...
    11x: picked up the new wireless headphones for runs and theyve be...
    10x: daily grind hitting hard this month with rent up 15 percent ...
    9x: booked a weekend getaway to the coast and the drive was brut...
    8x: the whole setup with the new router, performance is god tier...
    7x: just watched the season opener with friends, acting in the l...

[2] Duplicate text+aspect_term combinations:
  Unique combinations: 989
  Duplicated combination

In [10]:
#%%
# =======================================================================
# CELL 7: FINAL EXPORT FOR STAGE 1
# =======================================================================
print("="*70)
print("CELL 7: CREATING FINAL OUTPUTS")
print("="*70)
print(f"Time: {pd.Timestamp.now(tz='UTC')}\n")

# Load all data
print("[Step 1] Loading data...")
with open('candidates.json', 'r') as f:
    candidates = json.load(f)
with open('cluster_stats.json', 'r') as f:
    cluster_stats = json.load(f)

embeddings_matrix = np.load('embeddings.npy')
labels = np.load('cluster_labels.npy')
probabilities = np.load('cluster_probabilities.npy')

print(f"  ✓ All data loaded\n")

# Create final mapping
print("[Step 2] Creating cluster mapping...")

cluster_mapping = {
    "metadata": {
        "total_annotations": len(labels),
        "num_clusters": int(labels.max() + 1),
        "noise_count": int((labels == -1).sum()),
        "clustered_count": int((labels != -1).sum()),
        "silhouette_score": 0.568,  # From Cell 5
        "davies_bouldin_index": 0.893,  # From Cell 5
        "clustering_params": {
            "algorithm": "HDBSCAN",
            "min_cluster_size": 2,
            "min_samples": 1,
            "metric": "euclidean",
            "cluster_selection_method": "eom"
        },
        "embedding_model": "google/flan-t5-base",
        "adapter": "flan-t5-base-social-lora-masked-span",
        "created_at": pd.Timestamp.now(tz='UTC').isoformat()
    },
    "clusters": []
}

# Add noise cluster
noise_indices = [int(i) for i, label in enumerate(labels) if label == -1]
if noise_indices:
    cluster_mapping["clusters"].append({
        "cluster_id": -1,
        "canonical_aspect": "noise_singleton",
        "size": len(noise_indices),
        "member_ids": noise_indices,
        "representatives": []
    })

# Add real clusters
for stat in cluster_stats:
    # Determine canonical aspect (most common)
    if stat['aspect_terms']:
        canonical = stat['aspect_terms'][0][0]
    else:
        canonical = "unknown"
    
    cluster_mapping["clusters"].append({
        "cluster_id": stat['cluster_id'],
        "canonical_aspect": canonical,
        "size": stat['size'],
        "variance": stat['variance'],
        "aspect_term_distribution": dict(stat['aspect_terms']),
        "sentiment_distribution": stat.get('sentiment_distribution', {}),
        "implicit_ratio": stat.get('implicit_ratio', 0.0),
        "avg_probability": stat['avg_probability'],
        "member_ids": stat['member_indices'],
        "representatives": [
            {
                "text": rep["text"],
                "span_text": rep["span_text"],
                "aspect_term": rep["aspect_term"],
                "sentiment": rep["sentiment"]
            }
            for rep in stat['representatives'][:3]  # Top 3 only
        ]
    })

# Save cluster mapping
print("[Step 3] Saving cluster_mapping.json...")
with open("cluster_mapping.json", "w") as f:
    json.dump(cluster_mapping, f, indent=2)
print(f"  ✓ Saved cluster_mapping.json\n")

# Create summary report
print("[Step 4] Creating summary report...")

report = f"""
{'='*70}
CLUSTERING PIPELINE - FINAL REPORT (ARTIFACT A)
{'='*70}

EXECUTION DETAILS
-----------------
Timestamp: {pd.Timestamp.now(tz='UTC').isoformat()}
User: saheenus-pg
Total Samples: {len(labels)}

CLUSTERING RESULTS
------------------
Total Clusters: {labels.max() + 1}
Noise Points: {(labels == -1).sum()} ({(labels == -1).sum() / len(labels) * 100:.1f}%)
Clustered Points: {(labels != -1).sum()} ({(labels != -1).sum() / len(labels) * 100:.1f}%)

Cluster Size Distribution:
  Min: {min(s['size'] for s in cluster_stats)}
  Max: {max(s['size'] for s in cluster_stats)}
  Mean: {np.mean([s['size'] for s in cluster_stats]):.1f}
  Median: {np.median([s['size'] for s in cluster_stats]):.0f}

QUALITY METRICS
---------------
Silhouette Score: 0.568 ✅ EXCELLENT
Davies-Bouldin Index: 0.893 ✅ GOOD
Overall Assessment: PASS (3/4 quality gates)

QUALITY GATES
-------------
✅ Silhouette ≥ 0.30: PASS (0.568)
✅ Davies-Bouldin < 1.5: PASS (0.893)
⚠️  Cluster Count 60-150: MARGINAL (359)
⚠️  Noise < 20%: MARGINAL (20.5%)

OUTPUTS CREATED
---------------
1. cluster_mapping.json - Main output for Stage 1
2. cluster_stats.json - Detailed statistics
3. embeddings.npy - T5 embeddings (1193 x 768)
4. cluster_labels.npy - Cluster assignments
5. cluster_probabilities.npy - HDBSCAN soft assignments
6. candidates.json - Processed input data

NEXT STEPS
----------
✅ Clustering complete - ready for Stage 1: Contrastive Learning

Use cluster_mapping.json as input for:
- Positive pair mining (same cluster)
- Hard negative mining (different clusters, same aspect)
- Contrastive loss computation

{'='*70}
"""

with open("clustering_report.txt", "w") as f:
    f.write(report)

print(report)

print("  ✓ Saved clustering_report.txt\n")

print("="*70)
print("CELL 7: COMPLETE ✅")
print("="*70)
print("\n🎉 CLUSTERING PIPELINE COMPLETE!")
print("\nAll outputs saved:")
print("  • cluster_mapping.json (MAIN OUTPUT)")
print("  • cluster_stats.json")
print("  • clustering_report.txt")
print("  • embeddings.npy")
print("  • cluster_labels.npy")
print("  • cluster_probabilities.npy")
print("  • candidates.json")
print("\n✅ Ready for Stage 1: Contrastive Learning\n")

CELL 7: CREATING FINAL OUTPUTS
Time: 2025-11-09 07:51:00.422173+00:00

[Step 1] Loading data...
  ✓ All data loaded

[Step 2] Creating cluster mapping...
[Step 3] Saving cluster_mapping.json...
  ✓ Saved cluster_mapping.json

[Step 4] Creating summary report...

CLUSTERING PIPELINE - FINAL REPORT (ARTIFACT A)

EXECUTION DETAILS
-----------------
Timestamp: 2025-11-09T07:51:00.587391+00:00
User: saheenus-pg
Total Samples: 1193

CLUSTERING RESULTS
------------------
Total Clusters: 359
Noise Points: 244 (20.5%)
Clustered Points: 949 (79.5%)

Cluster Size Distribution:
  Min: 2
  Max: 11
  Mean: 2.6
  Median: 2

QUALITY METRICS
---------------
Silhouette Score: 0.568 ✅ EXCELLENT
Davies-Bouldin Index: 0.893 ✅ GOOD
Overall Assessment: PASS (3/4 quality gates)

QUALITY GATES
-------------
✅ Silhouette ≥ 0.30: PASS (0.568)
✅ Davies-Bouldin < 1.5: PASS (0.893)
⚠️  Cluster Count 60-150: MARGINAL (359)
⚠️  Noise < 20%: MARGINAL (20.5%)

OUTPUTS CREATED
---------------
1. cluster_mapping.json - M

# Contrastive learning

In [11]:
#%%
# =======================================================================
# CELL 8: PREPARE CONTRASTIVE LEARNING DATA
# =======================================================================
print("="*70)
print("CELL 8: PREPARING CONTRASTIVE PAIRS")
print("="*70)
print(f"Time: {pd.Timestamp.now(tz='UTC')}\n")

# Load data (already in memory from previous cells)
print("[Step 1] Loading clustering results...")
with open('candidates.json', 'r') as f:
    candidates = json.load(f)
with open('cluster_stats.json', 'r') as f:
    cluster_stats = json.load(f)

labels = np.load('cluster_labels.npy')
embeddings_matrix = np.load('embeddings.npy')

print(f"  ✓ Candidates: {len(candidates)}")
print(f"  ✓ Clusters: {len(cluster_stats)}")
print(f"  ✓ Embeddings: {embeddings_matrix.shape}\n")

# Build cluster membership index
print("[Step 2] Building cluster index...")
cluster_members = defaultdict(list)

for idx, label in enumerate(labels):
    if label != -1:  # Skip noise
        cluster_members[label].append(idx)

# Filter clusters by size (need at least 2 for positive pairs)
valid_clusters = {cid: members for cid, members in cluster_members.items() if len(members) >= 2}

print(f"  ✓ Valid clusters (≥2 members): {len(valid_clusters)}")
print(f"  ✓ Total samples in valid clusters: {sum(len(m) for m in valid_clusters.values())}\n")

# Mine positive pairs (same cluster)
print("[Step 3] Mining positive pairs...")
positive_pairs = []

for cluster_id, members in valid_clusters.items():
    # Sample all possible pairs within cluster
    for i in range(len(members)):
        for j in range(i + 1, len(members)):
            positive_pairs.append({
                "anchor_idx": members[i],
                "positive_idx": members[j],
                "cluster_id": cluster_id
            })

print(f"  ✓ Positive pairs: {len(positive_pairs)}\n")

# Mine hard negatives (different clusters, prefer same domain)
print("[Step 4] Mining hard negative pairs...")

# Group clusters by dominant aspect domain
def get_domain(aspect):
    """Map aspect to broad domain"""
    food_aspects = ['taste', 'freshness', 'portion', 'food']
    tech_aspects = ['battery', 'camera', 'performance', 'screen', 'design']
    service_aspects = ['service', 'wait', 'staff', 'responsiveness']
    entertainment_aspects = ['plot', 'acting', 'dialogue', 'music', 'soundtrack']
    policy_aspects = ['policy', 'healthcare', 'education', 'tax', 'election']
    
    aspect_lower = aspect.lower()
    
    if any(term in aspect_lower for term in food_aspects):
        return 'food'
    elif any(term in aspect_lower for term in tech_aspects):
        return 'tech'
    elif any(term in aspect_lower for term in service_aspects):
        return 'service'
    elif any(term in aspect_lower for term in entertainment_aspects):
        return 'entertainment'
    elif any(term in aspect_lower for term in policy_aspects):
        return 'policy'
    else:
        return 'other'

# Map clusters to domains
cluster_domains = {}
for stat in cluster_stats:
    cid = stat['cluster_id']
    if cid in valid_clusters:
        # Get most common aspect
        if stat['aspect_terms']:
            top_aspect = stat['aspect_terms'][0][0]
            cluster_domains[cid] = get_domain(top_aspect)

# Mine hard negatives (same domain, different cluster)
hard_negatives = []

for cluster_id, members in valid_clusters.items():
    domain = cluster_domains.get(cluster_id, 'other')
    
    # Find other clusters in same domain
    same_domain_clusters = [cid for cid, dom in cluster_domains.items() 
                           if cid != cluster_id and dom == domain]
    
    if same_domain_clusters:
        # Sample negatives from same domain
        for anchor_idx in members[:min(5, len(members))]:  # Limit to 5 per cluster
            # Sample from random same-domain cluster
            neg_cluster = random.choice(same_domain_clusters)
            neg_members = valid_clusters[neg_cluster]
            negative_idx = random.choice(neg_members)
            
            hard_negatives.append({
                "anchor_idx": anchor_idx,
                "negative_idx": negative_idx,
                "anchor_cluster": cluster_id,
                "negative_cluster": neg_cluster,
                "domain": domain
            })

print(f"  ✓ Hard negatives: {len(hard_negatives)}\n")

# Create training triplets
print("[Step 5] Creating training triplets...")
triplets = []

for pos_pair in positive_pairs:
    anchor_idx = pos_pair["anchor_idx"]
    positive_idx = pos_pair["positive_idx"]
    
    # Find a hard negative for this anchor
    anchor_cluster = pos_pair["cluster_id"]
    
    # Try to find same-domain negative
    matching_negatives = [hn for hn in hard_negatives 
                         if hn["anchor_idx"] == anchor_idx]
    
    if matching_negatives:
        neg = random.choice(matching_negatives)
        negative_idx = neg["negative_idx"]
    else:
        # Random negative from different cluster
        all_other_samples = [idx for cid, members in valid_clusters.items() 
                           if cid != anchor_cluster for idx in members]
        if all_other_samples:
            negative_idx = random.choice(all_other_samples)
        else:
            continue
    
    triplets.append({
        "anchor": anchor_idx,
        "positive": positive_idx,
        "negative": negative_idx
    })

print(f"  ✓ Triplets created: {len(triplets)}\n")

# Save triplets
with open("triplets.json", "w") as f:
    json.dump(triplets, f, indent=2)

print(f"  ✓ Saved triplets.json")

# Statistics
print("\n[Step 6] Triplet statistics:")
print(f"  Total triplets: {len(triplets)}")
print(f"  Positive pairs: {len(positive_pairs)}")
print(f"  Hard negatives: {len(hard_negatives)}")

# Domain distribution
domain_counts = Counter(cluster_domains.values())
print(f"\n  Domain distribution:")
for domain, count in domain_counts.most_common():
    print(f"    {domain}: {count} clusters")

print("\n" + "="*70)
print("CELL 8: COMPLETE ✅")
print("="*70)
print(f"\n✅ {len(triplets)} triplets ready for contrastive learning")
print("➡️  Next: Run Cell 9 (train with InfoNCE loss)\n")

CELL 8: PREPARING CONTRASTIVE PAIRS
Time: 2025-11-09 07:51:00.635854+00:00

[Step 1] Loading clustering results...
  ✓ Candidates: 1193
  ✓ Clusters: 359
  ✓ Embeddings: (1193, 768)

[Step 2] Building cluster index...
  ✓ Valid clusters (≥2 members): 359
  ✓ Total samples in valid clusters: 949

[Step 3] Mining positive pairs...
  ✓ Positive pairs: 1032

[Step 4] Mining hard negative pairs...
  ✓ Hard negatives: 921

[Step 5] Creating training triplets...
  ✓ Triplets created: 1032

  ✓ Saved triplets.json

[Step 6] Triplet statistics:
  Total triplets: 1032
  Positive pairs: 1032
  Hard negatives: 921

  Domain distribution:
    other: 223 clusters
    tech: 42 clusters
    service: 28 clusters
    food: 27 clusters
    entertainment: 23 clusters
    policy: 16 clusters

CELL 8: COMPLETE ✅

✅ 1032 triplets ready for contrastive learning
➡️  Next: Run Cell 9 (train with InfoNCE loss)



In [12]:
#%%
# =======================================================================
# CELL 9: CONTRASTIVE LEARNING WITH InfoNCE (FIXED GRADIENTS)
# =======================================================================
print("="*70)
print("CELL 9: TRAINING WITH InfoNCE LOSS (FROM DAPT CHECKPOINT)")
print("="*70)
print(f"Time: {pd.Timestamp.now(tz='UTC')}")
print("Estimated time: 10-15 minutes\n")

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, AutoTokenizer, get_linear_schedule_with_warmup
from peft import PeftModel, PeftConfig
from tqdm.auto import tqdm

# Configuration
BASE_MODEL_NAME = "google/flan-t5-base"
DAPT_ADAPTER_DIR = "/kaggle/input/t5-base-700k-masked-span/flan-t5-base-social-lora-masked-span"
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
NUM_EPOCHS = 3
TEMPERATURE = 0.07
MAX_LENGTH = 128

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}\n")

# Load data
print("[Step 1] Loading data...")
with open('candidates.json', 'r') as f:
    candidates = json.load(f)
with open('triplets.json', 'r') as f:
    triplets = json.load(f)

print(f"  ✓ Candidates: {len(candidates)}")
print(f"  ✓ Triplets: {len(triplets)}\n")

# Load DAPT model
print("[Step 2] Loading DAPT-adapted model...")
print(f"  Base: {BASE_MODEL_NAME}")
print(f"  DAPT adapter: {DAPT_ADAPTER_DIR}")

try:
    # Load PEFT config
    peft_config = PeftConfig.from_pretrained(DAPT_ADAPTER_DIR)
    print(f"  ✓ PEFT config loaded")
    
    # Load base model
    base_model = T5ForConditionalGeneration.from_pretrained(BASE_MODEL_NAME)
    print(f"  ✓ Base model loaded")
    
    # Load adapter
    model_with_adapter = PeftModel.from_pretrained(base_model, DAPT_ADAPTER_DIR)
    print(f"  ✓ Adapter loaded")
    
    # Merge adapter weights
    model = model_with_adapter.merge_and_unload()
    print(f"  ✅ DAPT adapter merged")
    
    # Delete references to free memory
    del base_model
    del model_with_adapter
    
except Exception as e:
    print(f"  ⚠️  Adapter loading failed: {e}")
    print(f"  Falling back to base model")
    model = T5ForConditionalGeneration.from_pretrained(BASE_MODEL_NAME)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)

# CRITICAL FIX: Enable gradients for all parameters
print(f"\n[Step 3] Enabling gradients...")
for param in model.parameters():
    param.requires_grad = True

model.to(device)
model.train()

# Verify gradients are enabled
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"  ✓ Trainable parameters: {trainable_params:,} / {total_params:,}")
print(f"  ✓ Model on device: {device}\n")

# Dataset
class TripletDataset(Dataset):
    def __init__(self, triplets, candidates):
        self.triplets = triplets
        self.candidates = candidates
    
    def __len__(self):
        return len(self.triplets)
    
    def __getitem__(self, idx):
        triplet = self.triplets[idx]
        
        anchor_text = self.candidates[triplet['anchor']]['span_with_context']
        positive_text = self.candidates[triplet['positive']]['span_with_context']
        negative_text = self.candidates[triplet['negative']]['span_with_context']
        
        return {
            'anchor': anchor_text,
            'positive': positive_text,
            'negative': negative_text
        }

# Create datasets
print("[Step 4] Creating datasets...")
dataset = TripletDataset(triplets, candidates)

# Split train/val (80/20)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

torch.manual_seed(42)
train_dataset, val_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size]
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"  ✓ Train samples: {len(train_dataset)}")
print(f"  ✓ Val samples: {len(val_dataset)}\n")

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_loader) * NUM_EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * total_steps),
    num_training_steps=total_steps
)

# InfoNCE Loss
class InfoNCELoss(nn.Module):
    def __init__(self, temperature=0.07):
        super().__init__()
        self.temperature = temperature
    
    def forward(self, anchor, positive, negative):
        # Normalize embeddings
        anchor = F.normalize(anchor, dim=1)
        positive = F.normalize(positive, dim=1)
        negative = F.normalize(negative, dim=1)
        
        # Compute similarities
        pos_sim = torch.sum(anchor * positive, dim=1) / self.temperature
        neg_sim = torch.sum(anchor * negative, dim=1) / self.temperature
        
        # InfoNCE loss
        logits = torch.stack([pos_sim, neg_sim], dim=1)
        labels = torch.zeros(logits.shape[0], dtype=torch.long, device=logits.device)
        
        loss = F.cross_entropy(logits, labels)
        
        return loss

criterion = InfoNCELoss(temperature=TEMPERATURE)

# Encoding function
def encode_batch(texts):
    inputs = tokenizer(
        texts,
        return_tensors="pt",
        max_length=MAX_LENGTH,
        truncation=True,
        padding="max_length"
    ).to(device)
    
    encoder = model.get_encoder()
    encoder_outputs = encoder(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask']
    )
    
    # Masked mean pooling
    hidden_states = encoder_outputs.last_hidden_state
    attention_mask = inputs['attention_mask'].unsqueeze(-1)
    masked_hidden = hidden_states * attention_mask
    sum_hidden = masked_hidden.sum(dim=1)
    sum_mask = attention_mask.sum(dim=1).clamp(min=1e-9)
    embeddings = sum_hidden / sum_mask
    
    return embeddings

# Training loop
print("[Step 5] Training...")
print(f"  Epochs: {NUM_EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Temperature: {TEMPERATURE}\n")

train_losses = []
val_losses = []

for epoch in range(NUM_EPOCHS):
    print(f"\n{'='*70}")
    print(f"EPOCH {epoch + 1}/{NUM_EPOCHS}")
    print(f"{'='*70}")
    
    # Training
    model.train()
    epoch_train_loss = 0
    
    pbar = tqdm(train_loader, desc=f"Training")
    for batch in pbar:
        optimizer.zero_grad()
        
        # Encode triplets
        anchor_emb = encode_batch(batch['anchor'])
        positive_emb = encode_batch(batch['positive'])
        negative_emb = encode_batch(batch['negative'])
        
        # Compute loss
        loss = criterion(anchor_emb, positive_emb, negative_emb)
        
        # Backward
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        epoch_train_loss += loss.item()
        pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    avg_train_loss = epoch_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    
    # Validation
    model.eval()
    epoch_val_loss = 0
    
    with torch.no_grad():
        pbar = tqdm(val_loader, desc=f"Validation")
        for batch in pbar:
            anchor_emb = encode_batch(batch['anchor'])
            positive_emb = encode_batch(batch['positive'])
            negative_emb = encode_batch(batch['negative'])
            
            loss = criterion(anchor_emb, positive_emb, negative_emb)
            epoch_val_loss += loss.item()
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    avg_val_loss = epoch_val_loss / len(val_loader)
    val_losses.append(avg_val_loss)
    
    print(f"\nEpoch {epoch + 1} Summary:")
    print(f"  Train Loss: {avg_train_loss:.4f}")
    print(f"  Val Loss:   {avg_val_loss:.4f}")

# Save model
print(f"\n[Step 6] Saving fine-tuned model...")
output_dir = "t5-dapt-contrastive-finetuned"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"  ✓ Saved to: {output_dir}\n")

# Save training history
history = {
    "train_losses": train_losses,
    "val_losses": val_losses,
    "config": {
        "base_model": BASE_MODEL_NAME,
        "dapt_adapter": DAPT_ADAPTER_DIR,
        "batch_size": BATCH_SIZE,
        "learning_rate": LEARNING_RATE,
        "epochs": NUM_EPOCHS,
        "temperature": TEMPERATURE,
        "train_samples": len(train_dataset),
        "val_samples": len(val_dataset)
    }
}

with open("training_history.json", "w") as f:
    json.dump(history, f, indent=2)

print(f"  ✓ Saved training_history.json")

# Summary
print("\n[Step 7] Training summary:")
print(f"  Final train loss: {train_losses[-1]:.4f}")
print(f"  Final val loss:   {val_losses[-1]:.4f}")
print(f"  Best val loss:    {min(val_losses):.4f} (epoch {val_losses.index(min(val_losses)) + 1})")

# Cleanup
del model
torch.cuda.empty_cache()

print("\n" + "="*70)
print("CELL 9: COMPLETE ✅")
print("="*70)
print(f"Time: {pd.Timestamp.now(tz='UTC')}")
print("\n✅ DAPT model fine-tuned with InfoNCE contrastive loss")
print("➡️  Next: Run Cell 10 (generate new embeddings)\n")

CELL 9: TRAINING WITH InfoNCE LOSS (FROM DAPT CHECKPOINT)
Time: 2025-11-09 07:51:00.878639+00:00
Estimated time: 10-15 minutes

Device: cuda

[Step 1] Loading data...
  ✓ Candidates: 1193
  ✓ Triplets: 1032

[Step 2] Loading DAPT-adapted model...
  Base: google/flan-t5-base
  DAPT adapter: /kaggle/input/t5-base-700k-masked-span/flan-t5-base-social-lora-masked-span
  ✓ PEFT config loaded
  ✓ Base model loaded
  ✓ Adapter loaded
  ✅ DAPT adapter merged

[Step 3] Enabling gradients...
  ✓ Trainable parameters: 247,577,856 / 247,577,856
  ✓ Model on device: cuda

[Step 4] Creating datasets...
  ✓ Train samples: 825
  ✓ Val samples: 207

[Step 5] Training...
  Epochs: 3
  Batch size: 16
  Learning rate: 2e-05
  Temperature: 0.07


EPOCH 1/3


Training:   0%|          | 0/52 [00:00<?, ?it/s]

Validation:   0%|          | 0/13 [00:00<?, ?it/s]


Epoch 1 Summary:
  Train Loss: 0.0837
  Val Loss:   0.0337

EPOCH 2/3


Training:   0%|          | 0/52 [00:00<?, ?it/s]

Validation:   0%|          | 0/13 [00:00<?, ?it/s]


Epoch 2 Summary:
  Train Loss: 0.0499
  Val Loss:   0.0244

EPOCH 3/3


Training:   0%|          | 0/52 [00:00<?, ?it/s]

Validation:   0%|          | 0/13 [00:00<?, ?it/s]


Epoch 3 Summary:
  Train Loss: 0.0404
  Val Loss:   0.0222

[Step 6] Saving fine-tuned model...
  ✓ Saved to: t5-dapt-contrastive-finetuned

  ✓ Saved training_history.json

[Step 7] Training summary:
  Final train loss: 0.0404
  Final val loss:   0.0222
  Best val loss:    0.0222 (epoch 3)

CELL 9: COMPLETE ✅
Time: 2025-11-09 07:54:28.257954+00:00

✅ DAPT model fine-tuned with InfoNCE contrastive loss
➡️  Next: Run Cell 10 (generate new embeddings)



# GENERATE NEW EMBEDDINGS (Compare Before/After)

In [13]:
#%%
# =======================================================================
# CELL 10: GENERATE NEW EMBEDDINGS WITH FINE-TUNED MODEL
# =======================================================================
print("="*70)
print("CELL 10: GENERATING NEW EMBEDDINGS")
print("="*70)
print(f"Time: {pd.Timestamp.now(tz='UTC')}")
print("Estimated time: 5-7 minutes\n")

from transformers import T5ForConditionalGeneration, AutoTokenizer

# Load data
print("[Step 1] Loading data...")
with open('candidates.json', 'r') as f:
    candidates = json.load(f)

print(f"  ✓ Candidates: {len(candidates)}\n")

# Load fine-tuned model
print("[Step 2] Loading fine-tuned model...")
MODEL_DIR = "t5-dapt-contrastive-finetuned"

model = T5ForConditionalGeneration.from_pretrained(MODEL_DIR)
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

print(f"  ✓ Model loaded from: {MODEL_DIR}")
print(f"  ✓ Device: {device}\n")

# Encoding function
def embed_batch(texts):
    inputs = tokenizer(
        texts,
        return_tensors="pt",
        max_length=128,
        truncation=True,
        padding="max_length"
    ).to(device)
    
    with torch.no_grad():
        encoder = model.get_encoder()
        encoder_outputs = encoder(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask']
        )
        
        # Masked mean pooling
        hidden_states = encoder_outputs.last_hidden_state
        attention_mask = inputs['attention_mask'].unsqueeze(-1)
        masked_hidden = hidden_states * attention_mask
        sum_hidden = masked_hidden.sum(dim=1)
        sum_mask = attention_mask.sum(dim=1).clamp(min=1e-9)
        embeddings = sum_hidden / sum_mask
        
        # L2 normalize
        embeddings = embeddings / torch.norm(embeddings, dim=1, keepdim=True).clamp(min=1e-9)
    
    return embeddings.cpu().numpy()

# Generate embeddings
print("[Step 3] Generating embeddings...")
batch_size = 32
texts_to_embed = [c['span_with_context'] for c in candidates]

print(f"  Total: {len(texts_to_embed)}")
print(f"  Batch size: {batch_size}\n")

all_embeddings = []
for i in range(0, len(texts_to_embed), batch_size):
    batch_texts = texts_to_embed[i:i + batch_size]
    batch_embeddings = embed_batch(batch_texts)
    all_embeddings.append(batch_embeddings)
    
    if (i // batch_size) % 10 == 0 or i + batch_size >= len(texts_to_embed):
        progress = min(i + batch_size, len(texts_to_embed))
        print(f"  {progress:4d} / {len(texts_to_embed)} ({progress/len(texts_to_embed)*100:5.1f}%)")

embeddings_new = np.vstack(all_embeddings)

# Save
np.save("embeddings_finetuned.npy", embeddings_new)
print(f"\n  ✓ Saved: embeddings_finetuned.npy")
print(f"  ✓ Shape: {embeddings_new.shape}\n")

# Compare with old embeddings
print("[Step 4] Comparing with original embeddings...")
embeddings_old = np.load("embeddings.npy")

print(f"  Old embeddings: {embeddings_old.shape}")
print(f"  New embeddings: {embeddings_new.shape}")

# Compute similarity change
print(f"\n  Sample similarity comparisons:")

# Load cluster labels to compare same-cluster vs different-cluster
labels = np.load('cluster_labels.npy')

# Find a few same-cluster pairs
same_cluster_pairs = []
for cid in range(min(10, labels.max())):
    members = np.where(labels == cid)[0]
    if len(members) >= 2:
        same_cluster_pairs.append((members[0], members[1]))
        if len(same_cluster_pairs) >= 5:
            break

# Find different-cluster pairs
diff_cluster_pairs = []
for i in range(5):
    idx1 = np.random.randint(0, len(labels))
    idx2 = np.random.randint(0, len(labels))
    if labels[idx1] != labels[idx2] and labels[idx1] != -1 and labels[idx2] != -1:
        diff_cluster_pairs.append((idx1, idx2))

print(f"\n  Same-cluster pair similarities (should increase):")
for i, (idx1, idx2) in enumerate(same_cluster_pairs, 1):
    sim_old = np.dot(embeddings_old[idx1], embeddings_old[idx2])
    sim_new = np.dot(embeddings_new[idx1], embeddings_new[idx2])
    delta = sim_new - sim_old
    print(f"    {i}. Old: {sim_old:.3f} → New: {sim_new:.3f} (Δ {delta:+.3f})")

print(f"\n  Different-cluster pair similarities (should decrease):")
for i, (idx1, idx2) in enumerate(diff_cluster_pairs, 1):
    sim_old = np.dot(embeddings_old[idx1], embeddings_old[idx2])
    sim_new = np.dot(embeddings_new[idx1], embeddings_new[idx2])
    delta = sim_new - sim_old
    print(f"    {i}. Old: {sim_old:.3f} → New: {sim_new:.3f} (Δ {delta:+.3f})")

# Aggregate statistics
same_sims_old = [np.dot(embeddings_old[i], embeddings_old[j]) for i, j in same_cluster_pairs]
same_sims_new = [np.dot(embeddings_new[i], embeddings_new[j]) for i, j in same_cluster_pairs]
diff_sims_old = [np.dot(embeddings_old[i], embeddings_old[j]) for i, j in diff_cluster_pairs]
diff_sims_new = [np.dot(embeddings_new[i], embeddings_new[j]) for i, j in diff_cluster_pairs]

print(f"\n  Aggregate improvements:")
print(f"    Same-cluster similarity:")
print(f"      Old: {np.mean(same_sims_old):.3f}")
print(f"      New: {np.mean(same_sims_new):.3f} ({np.mean(same_sims_new) - np.mean(same_sims_old):+.3f})")
print(f"    Different-cluster similarity:")
print(f"      Old: {np.mean(diff_sims_old):.3f}")
print(f"      New: {np.mean(diff_sims_new):.3f} ({np.mean(diff_sims_new) - np.mean(diff_sims_old):+.3f})")

separation_old = np.mean(same_sims_old) - np.mean(diff_sims_old)
separation_new = np.mean(same_sims_new) - np.mean(diff_sims_new)

print(f"\n  Cluster separation (same - diff):")
print(f"    Old: {separation_old:.3f}")
print(f"    New: {separation_new:.3f}")
print(f"    Improvement: {separation_new - separation_old:+.3f} ({(separation_new/separation_old - 1)*100:+.1f}%)")

if separation_new > separation_old:
    print(f"\n  ✅ Contrastive learning IMPROVED cluster separation!")
else:
    print(f"\n  ⚠️  Cluster separation did not improve significantly")

# Cleanup
del model
torch.cuda.empty_cache()

print("\n" + "="*70)
print("CELL 10: COMPLETE ✅")
print("="*70)
print(f"Time: {pd.Timestamp.now(tz='UTC')}")
print("\n✅ New embeddings generated with fine-tuned model")
print("➡️  Next: Run Cell 11 (final evaluation)\n")

CELL 10: GENERATING NEW EMBEDDINGS
Time: 2025-11-09 07:54:28.310777+00:00
Estimated time: 5-7 minutes

[Step 1] Loading data...
  ✓ Candidates: 1193

[Step 2] Loading fine-tuned model...
  ✓ Model loaded from: t5-dapt-contrastive-finetuned
  ✓ Device: cuda

[Step 3] Generating embeddings...
  Total: 1193
  Batch size: 32

    32 / 1193 (  2.7%)
   352 / 1193 ( 29.5%)
   672 / 1193 ( 56.3%)
   992 / 1193 ( 83.2%)
  1193 / 1193 (100.0%)

  ✓ Saved: embeddings_finetuned.npy
  ✓ Shape: (1193, 768)

[Step 4] Comparing with original embeddings...
  Old embeddings: (1193, 768)
  New embeddings: (1193, 768)

  Sample similarity comparisons:

  Same-cluster pair similarities (should increase):
    1. Old: 1.000 → New: 1.000 (Δ -0.000)
    2. Old: 1.000 → New: 1.000 (Δ +0.000)
    3. Old: 1.000 → New: 1.000 (Δ -0.000)
    4. Old: 1.000 → New: 1.000 (Δ +0.000)
    5. Old: 1.000 → New: 1.000 (Δ -0.000)

  Different-cluster pair similarities (should decrease):
    1. Old: 0.583 → New: 0.358 (Δ -0.2

# The fair evaluation comparing DAPT vs CL embeddings

In [14]:
#%%
# =======================================================================
# CELL 11: FAIR EVALUATION - DAPT vs DAPT+CL (REPRESENTATION LEVEL)
# =======================================================================
print("="*70)
print("CELL 11: COMPARING DAPT vs DAPT+CL EMBEDDINGS")
print("="*70)
print(f"Time: {pd.Timestamp.now(tz='UTC')}")
print("Estimated time: 15-20 minutes\n")

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from scipy.spatial.distance import cosine
from sklearn.metrics import roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

# Configuration
N_REPEATS = 20            # repeated few-shot trials
FEW_SHOT_K = 5            # shots per class
TEST_SIZE = 0.3
P_AT_K = [1, 5]
SIM_PAIR_SAMPLES = 2000   # number of same/different pairs
BOOTSTRAP_B = 1000
MIN_PER_CLASS = 8

# Load data
print("[Step 1] Loading embeddings...")
with open('candidates.json', 'r') as f:
    candidates = json.load(f)

embeddings_dapt = np.load('embeddings.npy')  # Original DAPT
embeddings_cl = np.load('embeddings_finetuned.npy')  # CL fine-tuned

labels = np.array([c['aspect_term'] for c in candidates])

print(f"  ✓ Samples: {len(candidates)}")
print(f"  ✓ DAPT embeddings: {embeddings_dapt.shape}")
print(f"  ✓ CL embeddings: {embeddings_cl.shape}")
print(f"  ✓ Unique aspects: {len(set(labels))}\n")

# Filter aspects with enough samples
from collections import Counter
aspect_counts = Counter(labels)
valid_aspects = [asp for asp, count in aspect_counts.items() if count >= MIN_PER_CLASS]

print(f"Aspects with ≥{MIN_PER_CLASS} samples: {len(valid_aspects)}")

# Filter data
valid_indices = [i for i, label in enumerate(labels) if label in valid_aspects]
X_dapt = embeddings_dapt[valid_indices]
X_cl = embeddings_cl[valid_indices]
y_full = labels[valid_indices]

print(f"Filtered samples: {len(y_full)}\n")

# =======================================================================
# EVALUATION 1: REPEATED FEW-SHOT CLASSIFICATION
# =======================================================================
print("="*70)
print("EVALUATION 1: FEW-SHOT ASPECT CLASSIFICATION")
print("="*70)
print(f"Testing: Can embeddings classify aspects with {FEW_SHOT_K} examples?\n")

def repeated_few_shot(embeddings, labels, k=5, repeats=20, test_size=0.3):
    """Run few-shot classification multiple times with different seeds"""
    accs = []
    f1s = []
    rng = np.random.RandomState(42)
    
    for seed in range(repeats):
        # Split
        X_train, X_test, y_train, y_test = train_test_split(
            embeddings, labels, test_size=test_size, random_state=seed, stratify=labels
        )
        
        # Sample k per class from training
        train_idx_by_class = {}
        for i, lab in enumerate(y_train):
            train_idx_by_class.setdefault(lab, []).append(i)
        
        chosen = []
        for lab, idxs in train_idx_by_class.items():
            rng.shuffle(idxs)
            chosen.extend(idxs[:k])
        
        if len(chosen) == 0:
            continue
        
        X_tr = X_train[chosen]
        y_tr = y_train[chosen]
        
        # Train classifier
        clf = LogisticRegression(max_iter=2000, multi_class='ovr', random_state=seed)
        clf.fit(X_tr, y_tr)
        y_pred = clf.predict(X_test)
        
        accs.append(accuracy_score(y_test, y_pred))
        f1s.append(f1_score(y_test, y_pred, average='macro'))
    
    return np.array(accs), np.array(f1s)

print(f"Running {N_REPEATS} trials with {FEW_SHOT_K}-shot learning...")
print("(This may take a few minutes...)\n")

accs_dapt, f1_dapt = repeated_few_shot(X_dapt, y_full, k=FEW_SHOT_K, repeats=N_REPEATS)
accs_cl, f1_cl = repeated_few_shot(X_cl, y_full, k=FEW_SHOT_K, repeats=N_REPEATS)

mean_acc_dapt, std_acc_dapt = accs_dapt.mean(), accs_dapt.std(ddof=1)
mean_acc_cl, std_acc_cl = accs_cl.mean(), accs_cl.std(ddof=1)

mean_f1_dapt, std_f1_dapt = f1_dapt.mean(), f1_dapt.std(ddof=1)
mean_f1_cl, std_f1_cl = f1_cl.mean(), f1_cl.std(ddof=1)

print(f"Results ({FEW_SHOT_K}-shot, {N_REPEATS} runs):")
print(f"  DAPT: acc={mean_acc_dapt:.3f} ± {std_acc_dapt:.3f}, macro-F1={mean_f1_dapt:.3f} ± {std_f1_dapt:.3f}")
print(f"  CL:   acc={mean_acc_cl:.3f} ± {std_acc_cl:.3f}, macro-F1={mean_f1_cl:.3f} ± {std_f1_cl:.3f}")

# Bootstrap CI for difference
print(f"\nBootstrap confidence interval for difference:")
diffs_acc = []
diffs_f1 = []
rng = np.random.RandomState(123)

for _ in range(BOOTSTRAP_B):
    i = rng.randint(0, N_REPEATS)
    j = rng.randint(0, N_REPEATS)
    diffs_acc.append(accs_cl[i] - accs_dapt[j])
    diffs_f1.append(f1_cl[i] - f1_dapt[j])

lo_acc, hi_acc = np.percentile(diffs_acc, [2.5, 97.5])
lo_f1, hi_f1 = np.percentile(diffs_f1, [2.5, 97.5])

print(f"  Accuracy difference: mean={np.mean(accs_cl - accs_dapt):.4f}, 95% CI=({lo_acc:.4f}, {hi_acc:.4f})")
print(f"  Macro-F1 difference: mean={np.mean(f1_cl - f1_dapt):.4f}, 95% CI=({lo_f1:.4f}, {hi_f1:.4f})")

if lo_acc > 0:
    print(f"  ✅ CL significantly better (CI excludes 0)")
else:
    print(f"  ⚠️  No significant difference (CI includes 0)")

# =======================================================================
# EVALUATION 2: RETRIEVAL METRICS (P@k, R@k)
# =======================================================================
print("\n" + "="*70)
print("EVALUATION 2: ASPECT RETRIEVAL")
print("="*70)
print("Testing: Can embeddings retrieve same-aspect neighbors?\n")

def precision_at_k(embs, labels, k=5):
    """Compute average precision at k for all samples"""
    sims = cosine_similarity(embs, embs)
    np.fill_diagonal(sims, -1.0)  # Exclude self
    idx = np.argsort(-sims, axis=1)[:, :k]
    precisions = []
    for i in range(len(embs)):
        precisions.append(np.mean([labels[i] == labels[j] for j in idx[i]]))
    return np.mean(precisions)

def recall_at_k(embs, labels, k=5):
    """Compute average recall at k for all samples"""
    sims = cosine_similarity(embs, embs)
    np.fill_diagonal(sims, -1.0)
    idx = np.argsort(-sims, axis=1)[:, :k]
    recalls = []
    for i in range(len(embs)):
        total_same = np.sum(labels == labels[i]) - 1  # Exclude self
        if total_same <= 0:
            continue
        recalls.append(np.sum([labels[j] == labels[i] for j in idx[i]]) / total_same)
    return np.mean(recalls)

print("Computing retrieval metrics...")
for k in P_AT_K:
    p_dapt = precision_at_k(X_dapt, y_full, k=k)
    p_cl = precision_at_k(X_cl, y_full, k=k)
    delta_p = p_cl - p_dapt
    print(f"  P@{k}: DAPT={p_dapt:.3f} | CL={p_cl:.3f} | Δ={delta_p:+.3f}")

r5_dapt = recall_at_k(X_dapt, y_full, k=5)
r5_cl = recall_at_k(X_cl, y_full, k=5)
delta_r = r5_cl - r5_dapt
print(f"  R@5: DAPT={r5_dapt:.3f} | CL={r5_cl:.3f} | Δ={delta_r:+.3f}")

# =======================================================================
# EVALUATION 3: SIMILARITY DISCRIMINATION (AUC)
# =======================================================================
print("\n" + "="*70)
print("EVALUATION 3: ASPECT SIMILARITY DISCRIMINATION")
print("="*70)
print(f"Testing: Can embeddings distinguish same vs different aspects?\n")

print(f"Sampling {SIM_PAIR_SAMPLES} pairs...")
valid_inds = np.arange(len(y_full))
same_pairs = []
diff_pairs = []
rng = np.random.RandomState(42)

# Sample pairs
for _ in range(SIM_PAIR_SAMPLES):
    # Same aspect
    lab = rng.choice(valid_aspects)
    inds_lab = np.where(y_full == lab)[0]
    if len(inds_lab) >= 2:
        i, j = rng.choice(inds_lab, 2, replace=False)
        same_pairs.append((i, j))
    
    # Different aspects
    a, b = rng.choice(valid_aspects, 2, replace=False)
    i = rng.choice(np.where(y_full == a)[0])
    j = rng.choice(np.where(y_full == b)[0])
    diff_pairs.append((i, j))

print(f"  Same-aspect pairs: {len(same_pairs)}")
print(f"  Different-aspect pairs: {len(diff_pairs)}\n")

def compute_sim_stats(embs):
    """Compute similarity statistics"""
    same = [1 - cosine(embs[i], embs[j]) for (i, j) in same_pairs]
    diff = [1 - cosine(embs[i], embs[j]) for (i, j) in diff_pairs]
    all_scores = same + diff
    y_true = [1]*len(same) + [0]*len(diff)
    auc = roc_auc_score(y_true, all_scores)
    sep = np.mean(same) - np.mean(diff)
    return np.mean(same), np.std(same), np.mean(diff), np.std(diff), sep, auc

sd_mean, sd_std, dd_mean, dd_std, sep_dapt, auc_dapt = compute_sim_stats(X_dapt)
sc_mean, sc_std, dc_mean, dc_std, sep_cl, auc_cl = compute_sim_stats(X_cl)

print(f"Results:")
print(f"  DAPT:")
print(f"    Same-aspect similarity:      {sd_mean:.3f} ± {sd_std:.3f}")
print(f"    Different-aspect similarity: {dd_mean:.3f} ± {dd_std:.3f}")
print(f"    Separation (same - diff):    {sep_dapt:.3f}")
print(f"    AUC (discrimination):        {auc_dapt:.3f}")

print(f"\n  CL:")
print(f"    Same-aspect similarity:      {sc_mean:.3f} ± {sc_std:.3f}")
print(f"    Different-aspect similarity: {dc_mean:.3f} ± {dc_std:.3f}")
print(f"    Separation (same - diff):    {sep_cl:.3f}")
print(f"    AUC (discrimination):        {auc_cl:.3f}")

print(f"\n  Improvements:")
print(f"    Separation: {sep_cl - sep_dapt:+.4f} ({(sep_cl/sep_dapt - 1)*100:+.1f}%)")
print(f"    AUC: {auc_cl - auc_dapt:+.4f} ({(auc_cl/auc_dapt - 1)*100:+.1f}%)")

# Permutation test for separation
print(f"\n  Permutation test for separation difference:")

def perm_test_sep(e1, e2, pairs_same, pairs_diff, n_perm=500):
    """Permutation test for separation difference"""
    def compute_sep(embs):
        same = np.array([1 - cosine(embs[i], embs[j]) for (i, j) in pairs_same])
        diff = np.array([1 - cosine(embs[i], embs[j]) for (i, j) in pairs_diff])
        return same.mean() - diff.mean()
    
    obs = compute_sep(e2) - compute_sep(e1)  # CL - DAPT
    combined = np.vstack([e1, e2])
    n = combined.shape[0] // 2
    cnt = 0
    rng = np.random.RandomState(42)
    
    for _ in range(n_perm):
        perm = rng.permutation(combined.shape[0])
        a = combined[perm[:n]]
        b = combined[perm[n:]]
        diff = compute_sep(a) - compute_sep(b)
        if abs(diff) >= abs(obs):
            cnt += 1
    
    pval = (cnt + 1) / (n_perm + 1)
    return obs, pval

obs_sep, pval_sep = perm_test_sep(X_dapt, X_cl, same_pairs, diff_pairs, n_perm=500)
print(f"    Observed difference: {obs_sep:.4f}")
print(f"    p-value: {pval_sep:.4f}")

if pval_sep < 0.05:
    print(f"    ✅ Statistically significant (p < 0.05)")
else:
    print(f"    ⚠️  Not significant (p ≥ 0.05)")

# =======================================================================
# FINAL SUMMARY TABLE
# =======================================================================
print("\n" + "="*70)
print("OVERALL EVALUATION SUMMARY")
print("="*70)

print(f"\n{'Metric':<30} {'DAPT':>12} {'DAPT+CL':>12} {'Δ':>10} {'Winner':>10}")
print("-" * 70)

# Few-shot classification
acc_winner = "CL ✅" if mean_acc_cl > mean_acc_dapt else "DAPT"
print(f"{'Few-shot Acc (5-shot)':<30} {mean_acc_dapt:>12.3f} {mean_acc_cl:>12.3f} {mean_acc_cl - mean_acc_dapt:>10.3f} {acc_winner:>10}")

f1_winner = "CL ✅" if mean_f1_cl > mean_f1_dapt else "DAPT"
print(f"{'Macro-F1 (5-shot)':<30} {mean_f1_dapt:>12.3f} {mean_f1_cl:>12.3f} {mean_f1_cl - mean_f1_dapt:>10.3f} {f1_winner:>10}")

# Retrieval
p1_dapt = precision_at_k(X_dapt, y_full, k=1)
p1_cl = precision_at_k(X_cl, y_full, k=1)
p1_winner = "CL ✅" if p1_cl > p1_dapt else "DAPT"
print(f"{'P@1':<30} {p1_dapt:>12.3f} {p1_cl:>12.3f} {p1_cl - p1_dapt:>10.3f} {p1_winner:>10}")

p5_dapt = precision_at_k(X_dapt, y_full, k=5)
p5_cl = precision_at_k(X_cl, y_full, k=5)
p5_winner = "CL ✅" if p5_cl > p5_dapt else "DAPT"
print(f"{'P@5':<30} {p5_dapt:>12.3f} {p5_cl:>12.3f} {p5_cl - p5_dapt:>10.3f} {p5_winner:>10}")

r5_winner = "CL ✅" if r5_cl > r5_dapt else "DAPT"
print(f"{'R@5':<30} {r5_dapt:>12.3f} {r5_cl:>12.3f} {r5_cl - r5_dapt:>10.3f} {r5_winner:>10}")

# Similarity
sep_winner = "CL ✅" if sep_cl > sep_dapt else "DAPT"
print(f"{'Separation (same - diff)':<30} {sep_dapt:>12.3f} {sep_cl:>12.3f} {sep_cl - sep_dapt:>10.3f} {sep_winner:>10}")

auc_winner = "CL ✅" if auc_cl > auc_dapt else "DAPT"
print(f"{'AUC (same/diff)':<30} {auc_dapt:>12.3f} {auc_cl:>12.3f} {auc_cl - auc_dapt:>10.3f} {auc_winner:>10}")

# Count wins
wins_cl = sum([
    mean_acc_cl > mean_acc_dapt,
    mean_f1_cl > mean_f1_dapt,
    p1_cl > p1_dapt,
    p5_cl > p5_dapt,
    r5_cl > r5_dapt,
    sep_cl > sep_dapt,
    auc_cl > auc_dapt
])

print("\n" + "="*70)
print("CONCLUSION")
print("="*70)

if wins_cl >= 6:
    print(f"✅✅✅ CL wins {wins_cl}/7 evaluations")
    print("   Contrastive Learning SIGNIFICANTLY IMPROVED aspect representations")
elif wins_cl >= 4:
    print(f"✅ CL wins {wins_cl}/7 evaluations")
    print("   Contrastive Learning IMPROVED aspect representations")
elif wins_cl >= 2:
    print(f"⚠️  CL wins {wins_cl}/7 evaluations")
    print("   Contrastive Learning shows MIXED results")
else:
    print(f"❌ CL wins {wins_cl}/7 evaluations")
    print("   Contrastive Learning did NOT improve representations")

print("\n" + "="*70)
print("CELL 11: COMPLETE ✅")
print("="*70)
print(f"Time: {pd.Timestamp.now(tz='UTC')}")
print("\n✅ Representation-level evaluation complete")
print("✅ This proves CL impact on encoder (independent of generation)")
print("\n📊 For thesis: Include the summary table above")
print("➡️  Next step: Stage 2 (ABSA fine-tuning) when data ready\n")

CELL 11: COMPARING DAPT vs DAPT+CL EMBEDDINGS
Time: 2025-11-09 07:54:39.578388+00:00
Estimated time: 15-20 minutes

[Step 1] Loading embeddings...
  ✓ Samples: 1193
  ✓ DAPT embeddings: (1193, 768)
  ✓ CL embeddings: (1193, 768)
  ✓ Unique aspects: 89

Aspects with ≥8 samples: 73
Filtered samples: 1107

EVALUATION 1: FEW-SHOT ASPECT CLASSIFICATION
Testing: Can embeddings classify aspects with 5 examples?

Running 20 trials with 5-shot learning...
(This may take a few minutes...)

Results (5-shot, 20 runs):
  DAPT: acc=0.421 ± 0.025, macro-F1=0.442 ± 0.026
  CL:   acc=0.444 ± 0.035, macro-F1=0.464 ± 0.038

Bootstrap confidence interval for difference:
  Accuracy difference: mean=0.0228, 95% CI=(-0.0631, 0.0931)
  Macro-F1 difference: mean=0.0222, 95% CI=(-0.0689, 0.1065)
  ⚠️  No significant difference (CI includes 0)

EVALUATION 2: ASPECT RETRIEVAL
Testing: Can embeddings retrieve same-aspect neighbors?

Computing retrieval metrics...
  P@1: DAPT=0.626 | CL=0.626 | Δ=+0.000
  P@5: DAPT